In [ ]:
import os
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer

from langchain_community.document_loaders import PyPDFLoader # 1.로드
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter # 2.청크
from langchain.embeddings import HuggingFaceEmbeddings # 3. 임베딩 모델
from langchain.vectorstores import FAISS # 3. 벡터 저장
from langchain.retrievers import BM25Retriever, EnsembleRetriever # 4. 검색 기법
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.docstore.document import Document

from transformers import pipeline

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# 생명보험
# 1. 문서 로드 및 청크 분할
loader = PyPDFLoader("한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=30
)
chunks = text_splitter.split_documents(documents)
chunks = chunks[11:]  # 불필요한 청크 제외 (예: 목차)

# 청크 생성 후 줄바꿈 처리
chunks = [
    Document(
        page_content=chunk.page_content.replace('\n', ' ').strip(), 
        metadata=chunk.metadata
    ) for chunk in chunks
]

# 각 청크에 ID 부여
for i, chunk in enumerate(chunks):
    chunk.metadata['doc_id'] = i

# 2. HuggingFace 임베딩 및 FAISS 생성
embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(documents=chunks, embedding=embedding)

# 3. 키워드 검색을 위한 BM25 생성
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 3

# 4. 의미 검색 + 키워드 검색 (앙상블)
top_k = 3
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": top_k})
ensemble_retrievers = [retriever, bm25_retriever]
ensemble_retriever = EnsembleRetriever(retrievers=ensemble_retrievers, weights=[0.7, 0.3])

# 5. LLM 초기화 및 RAG 체인 구성
llm = ChatOllama(model="llama3.1", temperature=0.8, num_predict=300)

retrieval_qa_chat_prompt = ChatPromptTemplate.from_template("""
다음 컨텍스트를 바탕으로 질문에 답변해주세요. 컨텍스트에 관련 정보가 없다면,
"주어진 정보로는 답변할 수 없습니다."라고 말씀해 주세요.

컨텍스트: {context}

질문: {input}

답변:
""")

combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(ensemble_retriever, combine_docs_chain)

In [7]:
# 손해보험
# 1. 문서 로드 및 청크 분할
loader = PyPDFLoader("한화 손해보험.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=30
)
chunks = text_splitter.split_documents(documents)
chunks = chunks[11:]  # 불필요한 청크 제외 (예: 목차)

# 청크 생성 후 줄바꿈 처리
chunks = [
    Document(
        page_content=chunk.page_content.replace('\n', ' ').strip(), 
        metadata=chunk.metadata
    ) for chunk in chunks
]

# 각 청크에 ID 부여
for i, chunk in enumerate(chunks):
    chunk.metadata['doc_id'] = i

# 2. HuggingFace 임베딩 및 FAISS 생성
embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(documents=chunks, embedding=embedding)

# 3. 키워드 검색을 위한 BM25 생성
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 3

# 4. 의미 검색 + 키워드 검색 (앙상블)
top_k = 3
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": top_k})
ensemble_retrievers = [retriever, bm25_retriever]
ensemble_retriever = EnsembleRetriever(retrievers=ensemble_retrievers, weights=[0.7, 0.3])

# 5. LLM 초기화 및 RAG 체인 구성
llm = ChatOllama(model="llama3.1", temperature=0.8, num_predict=300)

retrieval_qa_chat_prompt1 = ChatPromptTemplate.from_template("""
다음 컨텍스트를 바탕으로 질문에 답변해주세요. 컨텍스트에 관련 정보가 없다면,
"주어진 정보로는 답변할 수 없습니다."라고 말씀해 주세요.

컨텍스트: {context}

질문: {input}

답변:
""")

combine_docs_chain1 = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt1)
rag_chain1 = create_retrieval_chain(ensemble_retriever, combine_docs_chain1)

In [8]:
# 자동차보험
# 1. 문서 로드 및 청크 분할
loader = PyPDFLoader("한화 개인용 자동차보험.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=30
)
chunks = text_splitter.split_documents(documents)
chunks = chunks[11:]  # 불필요한 청크 제외 (예: 목차)

# 각 청크에 ID 부여
for i, chunk in enumerate(chunks):
    chunk.metadata['doc_id'] = i

# 2. HuggingFace 임베딩 및 FAISS 생성
embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(documents=chunks, embedding=embedding)

# 3. 키워드 검색을 위한 BM25 생성
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 3

# 4. 의미 검색 + 키워드 검색 (앙상블)
top_k = 3
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": top_k})
ensemble_retrievers = [retriever, bm25_retriever]
ensemble_retriever = EnsembleRetriever(retrievers=ensemble_retrievers, weights=[0.7, 0.3])

# 5. LLM 초기화 및 RAG 체인 구성
llm = ChatOllama(model="llama3.1", temperature=0.8, num_predict=300)

retrieval_qa_chat_prompt2 = ChatPromptTemplate.from_template("""
다음 컨텍스트를 바탕으로 질문에 답변해주세요. 컨텍스트에 관련 정보가 없다면,
"주어진 정보로는 답변할 수 없습니다."라고 말씀해 주세요.

컨텍스트: {context}

질문: {input}

답변:
""")

combine_docs_chain2 = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt2)
rag_chain2 = create_retrieval_chain(ensemble_retriever, combine_docs_chain2)

In [9]:
import gradio as gr

# 보험 종류별 RAG 체인 정의 (생성된 RAG 체인 설정)
rag_chain_cancer = rag_chain  # 생명보험 RAG 체인
rag_chain_life = rag_chain1  # 손해보험 RAG 체인
rag_chain_car = rag_chain2  # 자동차보험 RAG 체인

# 선택된 RAG 체인을 저장할 변수
selected_chain = {"current": None}

def set_rag_chain(chain_type):
    """선택된 RAG 체인을 설정하는 함수"""
    if chain_type == "생명보험":
        selected_chain["current"] = rag_chain_cancer
    elif chain_type == "손해보험":
        selected_chain["current"] = rag_chain_life
    elif chain_type == "자동차보험":
        selected_chain["current"] = rag_chain_car

def submit_question(question, chat_history):
    try:
        print(f"질문: {question}")  # 디버깅

        # 선택된 RAG 체인이 없으면 에러 반환
        if selected_chain["current"] is None:
            return chat_history, "먼저 보험 종류를 선택하세요.", []

        # 선택된 RAG 체인 실행
        result = selected_chain["current"].invoke({"input": question})
        print(f"RAG 체인 결과: {result}")  # 디버깅

        # 답변 및 상위 문서 가져오기
        answer = result.get("answer", "답변을 생성할 수 없습니다.")
        context_docs = result.get("context", [])[:3]  # 상위 3개 문서만 가져오기

        # 채팅 기록에 질문과 답변 추가
        chat_history.append((question, answer))

        # 하이라이트 데이터를 저장할 리스트
        highlight_parts = []

        for doc in context_docs:
            doc_text = doc.page_content
            doc_source = doc.metadata.get("source", "알 수 없음")
            doc_page = doc.metadata.get("page", "알 수 없음")

            # 문서를 나누고, 하이라이트된 문구만 처리
            sentences = doc_text.split('. ')
            for sentence in sentences:
                sentence = sentence.strip()
                if sentence and sentence in answer:  # 답변에 포함된 문장은 하이라이트 처리
                    highlight_parts.append((sentence, "highlight"))
                else:
                    highlight_parts.append((sentence, None))

            # 문서의 출처 정보는 별도로 추가
            highlight_parts.append((f"\n출처: {doc_source}, 페이지: {doc_page}\n", None))

        # Gradio로 올바르게 데이터 전달
        return chat_history, "", highlight_parts

    except Exception as e:
        print(f"오류 발생: {e}")  # 디버깅
        import traceback
        traceback.print_exc()
        return chat_history, f"오류 발생: {e}", []

# Gradio UI 구성
with gr.Blocks() as iface:
    # 제목 및 설명
    gr.Markdown("# 보험 문서 챗봇\n보험 문서에 대해 물어보면 답하는 챗봇입니다.")

    # PDF 버튼 및 챗봇 UI
    with gr.Row():
        cancer_btn = gr.Button("생명보험")
        life_btn = gr.Button("손해보험")
        car_btn = gr.Button("자동차보험")
        
    with gr.Row():  # 용어사전 버튼
        glossary_btn = gr.Button("용어사전 📖")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="뭐든지 물어보세요.", label="챗 입력")
    highlighted_text = gr.HighlightedText(label="문서 하이라이트", show_legend=True)

    # 버튼 행
    with gr.Row():
        submit_btn = gr.Button("보내기")
        retry_btn = gr.Button("다시보내기 ↩")
        undo_btn = gr.Button("이전 채팅 삭제 ❌")
        clear_btn = gr.Button("전체 채팅 삭제 💫")

    # 버튼 클릭 이벤트 설정
    def update_chatbot_with_message(chat_history, sender, message):
        chat_history.append((sender, message))  # 지정된 발신자와 메시지 추가
        return chat_history

    # 보험 선택 버튼 설정
    cancer_btn.click(
        lambda chat_history: (
            update_chatbot_with_message(chat_history, "생명보험", "생명보험 관련 정보를 물어보세요."),
            set_rag_chain("생명보험"),
        )[0],
        inputs=chatbot,
        outputs=chatbot,
    )
    life_btn.click(
        lambda chat_history: (
            update_chatbot_with_message(chat_history, "손해보험", "손해보험 관련 정보를 물어보세요."),
            set_rag_chain("손해보험"),
        )[0],
        inputs=chatbot,
        outputs=chatbot,
    )
    car_btn.click(
        lambda chat_history: (
            update_chatbot_with_message(chat_history, "자동차보험", "자동차보험 관련 정보를 물어보세요."),
            set_rag_chain("자동차보험"),
        )[0],
        inputs=chatbot,
        outputs=chatbot,
    )
    glossary_btn.click(
        lambda chat_history: update_chatbot_with_message(
            chat_history,
            "용어사전",
            (
                "용어사전에 대한 정보는 다음 링크를 참조하세요:\n"
                "- [보험 용어사전 1](https://www.hwgeneralins.com/lounge/dic/index.do)\n"
                "- [보험 용어사전 2](https://m.blog.naver.com/blogfsc/220149083103)"
            )
        ),
        inputs=chatbot,
        outputs=chatbot,
    )

    # 질문 제출 처리
    submit_btn.click(
        submit_question, 
        inputs=[msg, chatbot], 
        outputs=[chatbot, msg, highlighted_text]
    )
    
    # 다시 보내기 버튼
    retry_btn.click(lambda chat_history: chat_history, [chatbot], chatbot)

    # 이전 채팅 삭제 버튼
    undo_btn.click(lambda chat_history: chat_history[:-1] if chat_history else chat_history, [chatbot], chatbot)

    # 전체 채팅 삭제 버튼
    clear_btn.click(lambda: ([], ""), None, chatbot)  # 전체 대화 기록 삭제

iface.launch(share=True, debug=True)

/home/jonghai/.local/lib/python3.10/site-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://a8f1cf1d39cec0f32d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


질문: 보험금이 지급되는 경우는 뭐가 있어?
RAG 체인 결과: {'input': '보험금이 지급되는 경우는 뭐가 있어?', 'context': [Document(metadata={'source': '한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf', 'page': 7, 'doc_id': 4}, page_content='해지된  계약의  부활(효력회복 ))   보험금의  지급 보험금을  받을 수  있는지  궁금해요  제10조(보험금의  지급사유 ),  제12조(보험금을  지급하지  않는 사유)    보험금은  언제  지급되나요  제14조(보험금  등의 청구),  제15조(보험금  등의 지급절차 )'), Document(metadata={'source': '한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf', 'page': 167, 'doc_id': 553}, page_content='168 / 532 회사는  다음 중 한 가지로  보험금  지급사유가  발생한  때에는  보험금을  지급하지  않습니다 .  1. 피보험자 가 고의로  자신을  해친 경우  다만, 피보험자가  심신상실  등으로  자유로운  의사결정을  할 수 없는 상태에서  자신을  해침으로  인 하여 보험금  지급사유가  발생한  때에는  보험금을  지급하여  드립니다 .  2. 보험수익자가  고의로  피보험자를  해친 경우  다만, 그 보험수익자가  보험금의  일부 보험수익자인  경우에는  다른 보험수익자에  대한 보험금은   지급합니다 .  3. 계약자가  고의로  피보험자 를 해친 경우    제 2-8 조 사고증명서   ① 제1-3조(보험금  등의 청구) 제1항 제2호의 사고증명서는  ‘사망진단서 , 진단서 (병명기입 ) 수술확인서 (수 술기록지  등), 검사결과지 (조직검사 , 방사선검사  등) 등’을 말합니다 .'), Document(metadata={'source': 

In [10]:
import gradio as gr

# 보험 종류별 RAG 체인 정의 (생성된 RAG 체인 설정)
rag_chain_cancer = rag_chain  # 생명보험 RAG 체인
rag_chain_life = rag_chain1  # 손해보험 RAG 체인
rag_chain_car = rag_chain2  # 자동차보험 RAG 체인

# 선택된 RAG 체인을 저장할 변수
selected_chain = {"current": None}

def set_rag_chain(chain_type):
    """선택된 RAG 체인을 설정하는 함수"""
    if chain_type == "생명보험":
        selected_chain["current"] = rag_chain_cancer
    elif chain_type == "손해보험":
        selected_chain["current"] = rag_chain_life
    elif chain_type == "자동차보험":
        selected_chain["current"] = rag_chain_car

def submit_question(question, chat_history):
    try:
        print(f"질문: {question}")  # 디버깅

        # 선택된 RAG 체인이 없으면 에러 반환
        if selected_chain["current"] is None:
            return chat_history, "먼저 보험 종류를 선택하세요.", []

        # 선택된 RAG 체인 실행
        result = selected_chain["current"].invoke({"input": question})
        print(f"RAG 체인 결과: {result}")  # 디버깅

        # 답변 및 상위 문서 가져오기
        answer = result.get("answer", "답변을 생성할 수 없습니다.")
        context_docs = result.get("context", [])[:3]  # 상위 3개 문서만 가져오기

        # 채팅 기록에 질문과 답변 추가
        chat_history.append((question, answer))

        # 하이라이트 데이터를 저장할 리스트
        highlight_parts = []

        for doc in context_docs:
            doc_text = doc.page_content
            doc_source = doc.metadata.get("source", "알 수 없음")
            doc_page = doc.metadata.get("page", "알 수 없음")

            # 문서를 단어 단위로 나누고, 하이라이트된 단어만 처리
            words = doc_text.split()  # 단어 단위로 나누기
            for word in words:
                if word.strip() in answer:  # 답변에 포함된 단어는 하이라이트 처리
                    highlight_parts.append((word, "highlight"))
                else:  # 나머지는 일반 텍스트로 표시
                    highlight_parts.append((word, None))

            # 문서의 출처 정보는 별도로 추가
            highlight_parts.append((f"\n출처: {doc_source}, 페이지: {doc_page}\n", None))

        # Gradio로 올바르게 데이터 전달
        return chat_history, "", highlight_parts

    except Exception as e:
        print(f"오류 발생: {e}")  # 디버깅
        import traceback
        traceback.print_exc()
        return chat_history, f"오류 발생: {e}", []

# Gradio UI 구성
with gr.Blocks() as iface:
    # 제목 및 설명
    gr.Markdown("# 보험 문서 챗봇\n보험 문서에 대해 물어보면 답하는 챗봇입니다.")

    # PDF 버튼 및 챗봇 UI
    with gr.Row():
        cancer_btn = gr.Button("생명보험")
        life_btn = gr.Button("손해보험")
        car_btn = gr.Button("자동차보험")
        
    with gr.Row():  # 용어사전 버튼
        glossary_btn = gr.Button("용어사전 📖")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="뭐든지 물어보세요.", label="챗 입력")
    highlighted_text = gr.HighlightedText(label="문서 하이라이트", show_legend=True)

    # 버튼 행
    with gr.Row():
        submit_btn = gr.Button("보내기")
        retry_btn = gr.Button("다시보내기 ↩")
        undo_btn = gr.Button("이전 채팅 삭제 ❌")
        clear_btn = gr.Button("전체 채팅 삭제 💫")

    # 버튼 클릭 이벤트 설정
    def update_chatbot_with_message(chat_history, sender, message):
        chat_history.append((sender, message))  # 지정된 발신자와 메시지 추가
        return chat_history

    # 보험 선택 버튼 설정
    cancer_btn.click(
        lambda chat_history: (
            update_chatbot_with_message(chat_history, "생명보험", "생명보험 관련 정보를 물어보세요."),
            set_rag_chain("생명보험"),
        )[0],
        inputs=chatbot,
        outputs=chatbot,
    )
    life_btn.click(
        lambda chat_history: (
            update_chatbot_with_message(chat_history, "손해보험", "손해보험 관련 정보를 물어보세요."),
            set_rag_chain("손해보험"),
        )[0],
        inputs=chatbot,
        outputs=chatbot,
    )
    car_btn.click(
        lambda chat_history: (
            update_chatbot_with_message(chat_history, "자동차보험", "자동차보험 관련 정보를 물어보세요."),
            set_rag_chain("자동차보험"),
        )[0],
        inputs=chatbot,
        outputs=chatbot,
    )
    glossary_btn.click(
        lambda chat_history: update_chatbot_with_message(
            chat_history,
            "용어사전",
            (
                "용어사전에 대한 정보는 다음 링크를 참조하세요:\n"
                "- [보험 용어사전 1](https://www.hwgeneralins.com/lounge/dic/index.do)\n"
                "- [보험 용어사전 2](https://m.blog.naver.com/blogfsc/220149083103)"
            )
        ),
        inputs=chatbot,
        outputs=chatbot,
    )

    # 질문 제출 처리
    submit_btn.click(
        submit_question, 
        inputs=[msg, chatbot], 
        outputs=[chatbot, msg, highlighted_text]
    )
    
    # 다시 보내기 버튼
    retry_btn.click(lambda chat_history: chat_history, [chatbot], chatbot)

    # 이전 채팅 삭제 버튼
    undo_btn.click(lambda chat_history: chat_history[:-1] if chat_history else chat_history, [chatbot], chatbot)

    # 전체 채팅 삭제 버튼
    clear_btn.click(lambda: ([], ""), None, chatbot)  # 전체 대화 기록 삭제

iface.launch(share=True, debug=True)

/home/jonghai/.local/lib/python3.10/site-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://bfd471156f11b3f2ca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


질문: 보험금이 지급되는 경우는 뭐가 있어?
RAG 체인 결과: {'input': '보험금이 지급되는 경우는 뭐가 있어?', 'context': [Document(metadata={'source': '한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf', 'page': 7, 'doc_id': 4}, page_content='해지된  계약의  부활(효력회복 ))   보험금의  지급 보험금을  받을 수  있는지  궁금해요  제10조(보험금의  지급사유 ),  제12조(보험금을  지급하지  않는 사유)    보험금은  언제  지급되나요  제14조(보험금  등의 청구),  제15조(보험금  등의 지급절차 )'), Document(metadata={'source': '한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf', 'page': 167, 'doc_id': 553}, page_content='168 / 532 회사는  다음 중 한 가지로  보험금  지급사유가  발생한  때에는  보험금을  지급하지  않습니다 .  1. 피보험자 가 고의로  자신을  해친 경우  다만, 피보험자가  심신상실  등으로  자유로운  의사결정을  할 수 없는 상태에서  자신을  해침으로  인 하여 보험금  지급사유가  발생한  때에는  보험금을  지급하여  드립니다 .  2. 보험수익자가  고의로  피보험자를  해친 경우  다만, 그 보험수익자가  보험금의  일부 보험수익자인  경우에는  다른 보험수익자에  대한 보험금은   지급합니다 .  3. 계약자가  고의로  피보험자 를 해친 경우    제 2-8 조 사고증명서   ① 제1-3조(보험금  등의 청구) 제1항 제2호의 사고증명서는  ‘사망진단서 , 진단서 (병명기입 ) 수술확인서 (수 술기록지  등), 검사결과지 (조직검사 , 방사선검사  등) 등’을 말합니다 .'), Document(metadata={'source': 